In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
import pickle


import psutil
psutil.cpu_count()
p = psutil.Process()
p.cpu_affinity([0,1,2,3,4,5,6,7,8])

os.chdir(r'H:\all\RL_Shrinkage_2024')
from helpers import helper_functions as hf
from ONE_YR.NonLinear_Shrinkage import regression_evaluation_funcs as re_hf
from helpers import eval_function_new



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\traitlets\con

Idea: We have the future return matrices available and we know what stocks are in the universe at each given time point. We need to extract the market capitalization for the stocks at each rebalancing in the out of sample period.

In [2]:
## load permnos, and extract the dates for which we need the market caps

PF_SIZE = 30
base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
# IMPORT SHRK DATASETS
pf_size = PF_SIZE  # DONT CHANGE HERE!!
permnos = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\permnos_1Y_p{pf_size}.pickle")
rets_full = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\returns_full_1Y_p{pf_size}.pickle")
market_caps = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\oos_rebdate_marketcaps_1Y_p{pf_size}.pickle")

In [3]:
permnos.index = pd.to_datetime(permnos.index, format="%Y%m%d")
rebdates = permnos.index[list(range(5040, permnos.shape[0], 21))]
assert len(rebdates) == 253, "Rebdates is not of correct length.."
market_caps.index = pd.to_datetime(market_caps.index, format="%Y%m%d")

In [4]:
for rebdate in market_caps.index:
    market_caps.loc[market_caps.index == rebdate, ~market_caps.columns.isin(permnos.loc[rebdate,:])] = pd.NA

VW_weights = market_caps.T / market_caps.sum(axis=1)
VW_weights = VW_weights.T

EW_weights = market_caps.copy()
EW_weights[~EW_weights.isna()] = 1/PF_SIZE

In [5]:
## load permnos, and extract the dates for which we need the market caps
base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
# IMPORT SHRK DATASETS

VW_RES_DICT, EW_RES_DICT = {}, {}
#for PF_SIZE in [30, 50, 100, 225, 500]:
for PF_SIZE in [30]:
    pf_size = PF_SIZE  # DONT CHANGE HERE!!
    permnos = pd.read_pickle(
        fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\permnos_1Y_p{pf_size}.pickle")
    rets_full = pd.read_pickle(
        fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\returns_full_1Y_p{pf_size}.pickle")
    market_caps = pd.read_pickle(
        fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\oos_rebdate_marketcaps_1Y_p{pf_size}.pickle")

    permnos.index = pd.to_datetime(permnos.index, format="%Y%m%d")
    rebdates = permnos.index[list(range(5040, permnos.shape[0], 21))]
    assert len(rebdates) == 253, "Rebdates is not of correct length.."
    market_caps.index = pd.to_datetime(market_caps.index, format="%Y%m%d")

    for rebdate in market_caps.index:
        market_caps.loc[market_caps.index == rebdate, ~market_caps.columns.isin(permnos.loc[rebdate,:])] = pd.NA

    VW_weights = market_caps.T / market_caps.sum(axis=1)
    VW_weights = VW_weights.T

    EW_weights = market_caps.copy()
    EW_weights[~EW_weights.isna()] = 1/PF_SIZE

    # iterate through the return matrices
    VW_rawres = []
    EW_rawres = []
    reb_date_1 = permnos.index[0]
    add_idx = np.where(pd.to_datetime(rets_full.index, format="%Y%m%d") == reb_date_1)[0][0]
    for weights_idx, idx in enumerate(range(5040, permnos.shape[0], 21)):
        #past_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx - 21 * 12 * 1: idx + add_idx, :]
        #past_ret_mat = past_ret_mat.sub(past_ret_mat.mean())
        #past_ret_mat = past_ret_mat.fillna(0)
        fut_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx: idx + add_idx + 21, :]
        VW_rawres += (fut_ret_mat @ VW_weights.iloc[weights_idx, :].loc[permnos.iloc[idx]].T).tolist()
        EW_rawres += (fut_ret_mat @ EW_weights.iloc[weights_idx, :].loc[permnos.iloc[idx]].T).tolist()
    
    VW_RES_DICT[PF_SIZE] = VW_rawres
    EW_RES_DICT[PF_SIZE] = EW_rawres
        

In [17]:
(np.mean(EW_RES_DICT[30]) * (252) * 100) / (np.std(VW_RES_DICT[30]) * np.sqrt(252) * 100)

np.float64(0.5118799732335056)

In [16]:
np.mean(EW_RES_DICT[30]) * (252) * 100

np.float64(9.79134071146245)

In [129]:
VW_RES_DF = pd.DataFrame(VW_RES_DICT)
VW_RES_DF.index = permnos.index[5040:]

EW_RES_DF = pd.DataFrame(EW_RES_DICT)
EW_RES_DF.index = permnos.index[5040:]

VW_RES_DF.to_csv(r"H:\all\RL_Shrinkage_2024\ONE_YR\preprocessing\rets_permnos_1Y\VW_portfolio_daily_returns.csv")
EW_RES_DF.to_csv(r"H:\all\RL_Shrinkage_2024\ONE_YR\preprocessing\rets_permnos_1Y\EW_portfolio_daily_returns.csv")

In [185]:
# create matrix containing SD's and AV's for convex combinations

RES = {}
RES['AV'], RES['SD'] = {}, {}
for PF_SIZE in [30, 50, 100, 225, 500]:
    RES['SD'][PF_SIZE] = EW_RES_DF[PF_SIZE].std() * np.sqrt(252) * 100
    RES['AV'][PF_SIZE] = EW_RES_DF[PF_SIZE].mean() * (252) * 100

RES_DF = pd.DataFrame(RES)
RES_DF['IR'] = RES_DF['AV'] / RES_DF['SD']

RES_DF.round(2)

,AV,SD,IR
30,9.79,18.72,0.52
50,9.88,18.86,0.52
100,9.97,19.51,0.51
225,10.90,19.87,0.55
500,12.01,20.62,0.58


In [230]:
ints.round(2)['shrk_factor'].values

array([0.07, 0.07, 0.07, ..., 0.06, 0.06, 0.06])

In [234]:
pf_size=500
with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\fixed_shrkges_rawres_cov1Para_p{pf_size}.pickle", 'rb') as f:
    rawres_fixed_shrk_data = pickle.load(f)

ints = pd.read_csv(rf"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results\p500\c1p_p500_intensities.csv", index_col=0)


ints.round(2)

np.diag(rawres_fixed_shrk_data.loc[:, ints.round(2)['shrk_factor'].values])[5040:].std() * np.sqrt(252)*100

np.float64(11.013846795392254)

In [214]:
c1p = []
for pf_size in [30,50,100,225,500]:
    x = pd.read_csv(rf"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results\p{pf_size}\c1p_p{pf_size}_daily_returns.csv")
    c1p.append(x)



In [216]:
pd.DataFrame(c1p, columns=[30,50,100,225,500])

ValueError: Must pass 2-d input. shape=(5, 10353, 6)

In [206]:
c1p = pd.read_csv(r"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results\cov1para_daily_returns.csv", index_col=0)

c1p_res = {}

c1p_res["AV"] = c1p.iloc[5040:, :].mean() * (252)
c1p_res["SD"] = c1p.iloc[5040:, :].std() * np.sqrt(252) 
c1p_res["IR"] = c1p_res["AV"]  / c1p_res["SD"]

pd.DataFrame(c1p_res)

,AV,SD,IR
30,29.061739,6.651737,4.369045
50,27.840844,7.035900,3.956970
100,19.018341,6.825734,2.786270
225,23.284861,6.417609,3.628277
500,26.543277,6.240346,4.253494


In [186]:
# create matrix containing SD's and AV's for convex combinations

RES = {}
RES['AV'], RES['SD'] = {}, {}
for PF_SIZE in [30, 50, 100, 225, 500]:
    RES['SD'][PF_SIZE] = VW_RES_DF[PF_SIZE].std() * np.sqrt(252) * 100
    RES['AV'][PF_SIZE] = VW_RES_DF[PF_SIZE].mean() * (252) * 100

RES_DF = pd.DataFrame(RES)
RES_DF['IR'] = RES_DF['AV'] / RES_DF['SD']

RES_DF.round(2)

,AV,SD,IR
30,-34.14,317.04,-0.11
50,10.05,19.04,0.53
100,10.04,19.20,0.52
225,10.35,19.32,0.54
500,10.71,19.52,0.55


Summary of the EW and VW out of sample results

#### Calculate Portfolio Turnover Metrics for EW and VW portfolios

In [144]:
def compute_portfolio_stats(df):
    """
    Compute:
      1) TO = average (monthly) turnover
      2) GL = average (monthly) gross leverage
      3) PL = average (monthly) proportion of leverage (fraction of short positions)
    """
    
    # 1. Fill NaNs with zero
    df_filled = df.fillna(0)
    
    # 2. Compute average turnover (TO)
    #
    #    Turnover at date h is ||w_{h+1} - w_h||_1, i.e. sum of absolute differences
    #    across all stocks. Then we average over h = 1..(T-1).
    #
    #    diff() yields row_{h+1} - row_h for each column; then we take abs() and sum across columns.
    #    We skip the first NaN in diff() by taking [1:] and finally compute the mean.
    
    turnover_series = df_filled.diff().abs().sum(axis=1)      # || w_{h+1} - w_h ||_1 for each row
    TO = turnover_series[1:].mean()                           # average over h=1..T-1
    
    # 3. Compute average gross leverage (GL)
    #
    #    Gross leverage at date h is ||w_h||_1, i.e. the sum of absolute values of the weights on date h.
    #    We then average over all dates h.
    
    gross_leverage_series = df_filled.abs().sum(axis=1)       # || w_h ||_1 for each row
    GL = gross_leverage_series.mean()                         # average over all rows
    
    # 4. Compute average proportion of leverage (PL)
    #
    #    Proportion of leverage is the fraction of negative weights (short positions). 
    #    That is 1/(T * p) * sum_{h=1}^T sum_{i=1}^p 1{ w_{i,h} < 0 }, where p=#stocks, T=#dates.
    
    # total number of negative entries:
    num_negative = (df_filled < 0).sum().sum()
    # total number of "positions" = T * p:
    total_positions = df_filled.shape[0] * df_filled.shape[1]
    PL = num_negative / total_positions
    
    return TO, GL, PL

In [166]:
VW_results, EW_results = {}, {}
for PF_SIZE in [30, 50, 100, 225, 500]:
    pf_size = PF_SIZE  # DONT CHANGE HERE!!
    permnos = pd.read_pickle(
        fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\permnos_1Y_p{pf_size}.pickle")
    rets_full = pd.read_pickle(
        fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\returns_full_1Y_p{pf_size}.pickle")
    market_caps = pd.read_pickle(
        fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\oos_rebdate_marketcaps_1Y_p{pf_size}.pickle")

    permnos.index = pd.to_datetime(permnos.index, format="%Y%m%d")
    rebdates = permnos.index[list(range(5040, permnos.shape[0], 21))]
    assert len(rebdates) == 253, "Rebdates is not of correct length.."
    market_caps.index = pd.to_datetime(market_caps.index, format="%Y%m%d")

    for rebdate in market_caps.index:
        market_caps.loc[market_caps.index == rebdate, ~market_caps.columns.isin(permnos.loc[rebdate,:])] = pd.NA

    VW_weights = market_caps.T / market_caps.sum(axis=1)
    VW_weights = VW_weights.T

    EW_weights = market_caps.copy()
    EW_weights[~EW_weights.isna()] = 1/PF_SIZE

    TO, GL, PL = compute_portfolio_stats(VW_weights)
    VW_results[PF_SIZE] = [TO, GL, PL]

    TO, GL, PL = compute_portfolio_stats(EW_weights)
    EW_results[PF_SIZE] = [TO, GL, PL]

EW and VW Turnover Numbers:

In [167]:
EW_results_df = pd.DataFrame(EW_results, index=["TO", "GL", "PL"])
EW_results_df.T.round(2)

,TO,GL,PL
30,0.06,1.0,0.0
50,0.06,1.0,0.0
100,0.06,1.0,0.0
225,0.05,1.0,0.0
500,0.05,1.0,0.0


In [187]:
VW_results_df = pd.DataFrame(VW_results, index=["TO", "GL", "PL"])
VW_results_df.T.round(2)

,TO,GL,PL
30,0.07,1.0,0.0
50,0.07,1.0,0.0
100,0.07,1.0,0.0
225,0.06,1.0,0.0
500,0.06,1.0,0.0
